## 1. Чтение/запись в словарь. При попытке чтения несуществующего ключа возвращаетсяNone.

In [1]:
def fill_dict(keys,values):
    d=dict()
    for i in range(len(keys)):
        d[keys[i]]=values[i]
    return d

In [2]:
d=fill_dict(range(1000),range(1000))

In [3]:
%load_ext line_profiler

In [4]:
%lprun -f fill_dict fill_dict(range(10000000),range(10000000))

### Total time: 9.36776 s

In [5]:
def fill_dict1(keys,values):
    d={keys[i]:values[i] for i in range(len(keys))}
    return d

In [6]:
%lprun -f fill_dict1 fill_dict1(range(10000000),range(10000000))

### Total time: 4.19118 s

In [7]:
def fill_dict2(keys,values):
    d=dict(zip(keys,values))
    return d

In [8]:
%lprun -f fill_dict2 fill_dict2(range(10000000),range(10000000))

### Total time: 0.909937 s

In [9]:
%load_ext memory_profiler

In [10]:
%%writefile compare_fill_dict.py
from memory_profiler import profile

@profile
def compare_fill_dict():
    keys,values=range(100000),range(100000)
    d=dict()
    for i in range(len(keys)):
        d[keys[i]]=values[i]
    d1={keys[i]:values[i] for i in range(len(keys))}
    d2=dict(zip(keys,values))
    return d,d1,d2

if __name__=='__main__':
    compare_fill_dict()

Overwriting compare_fill_dict.py


In [11]:
!python compare_fill_dict.py

Filename: compare_fill_dict.py

Line #    Mem usage    Increment   Line Contents
     3     39.9 MiB     39.9 MiB   @profile
     4                             def compare_fill_dict():
     5     39.9 MiB      0.0 MiB       keys,values=range(100000),range(100000)
     6     39.9 MiB      0.0 MiB       d=dict()
     7     51.1 MiB      0.1 MiB       for i in range(len(keys)):
     8     51.1 MiB      2.5 MiB           d[keys[i]]=values[i]
     9     62.2 MiB      2.5 MiB       d1={keys[i]:values[i] for i in range(len(keys))}
    10     73.4 MiB     11.2 MiB       d2=dict(zip(keys,values))
    11     73.4 MiB      0.0 MiB       return d,d1,d2




### Встроенные функции обработки работают сильно быстрее, но жрут память, так как используют какие-то временные переменные

## 2. К каждому элементу списка применить какуе-либо преобразование (например, длячислового списка - возвести в кавдрат, для строкового - привести к верхнемурегистру, отфильтровать определенные символы, и т.д.).

In [12]:
def pow_list():
    a=[]
    for i in list(range(10000000)):
        a.append(1.000001**i)
    return a
        

In [13]:
%lprun -f pow_list pow_list()

### Total time: 9.58203 s

In [14]:
def pow_list1():
    return [1.000001**i for i in list(range(10000000))]

In [15]:
%lprun -f pow_list1 pow_list1()

### Total time: 3.3767 s

In [16]:
def pow_list2():
    return list(map(lambda i:1.000001**i,list(range(10000000))))

In [17]:
%lprun -f pow_list2 pow_list2()

### Total time: 4.94018 s

In [18]:
%%writefile compare_pow_list.py
from memory_profiler import profile

@profile
def compare_pow_list():
    a=[]
    for i in list(range(10000)):
        a.append(1.000001**i)
    a1=[1.000001**i for i in list(range(10000))]
    a2=list(map(lambda i:1.000001**i,list(range(10000))))
    

if __name__=='__main__':
    compare_pow_list()

Overwriting compare_pow_list.py


In [19]:
!python compare_pow_list.py

Filename: compare_pow_list.py

Line #    Mem usage    Increment   Line Contents
     3     39.9 MiB     39.9 MiB   @profile
     4                             def compare_pow_list():
     5     39.9 MiB      0.0 MiB       a=[]
     6     40.8 MiB      0.3 MiB       for i in list(range(10000)):
     7     40.8 MiB      0.1 MiB           a.append(1.000001**i)
     8     41.1 MiB      0.1 MiB       a1=[1.000001**i for i in list(range(10000))]
     9     41.6 MiB      0.1 MiB       a2=list(map(lambda i:1.000001**i,list(range(10000))))




### Преобразование map->list занимает довольно продолжительное время

## 3. Отсортировать список.

In [126]:
from numpy.random import randint
a=randint(0,400,5000)
a[:10]

array([ 62, 178, 381, 344, 226, 397,  98, 204,  30,  68])

In [127]:
import copy

In [128]:
def sort_buble(a):
    b=copy.deepcopy(a)
    for i in range(len(b)-1):
        for j in range(i+1,len(b)):
            if b[i]>b[j]:
                b[i],b[j]=b[j],b[i]
    return b

In [147]:
print(*(sort_buble(a)[:50]))

0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3


In [132]:
%lprun -f sort_buble sort_buble(a)

### Total time: 11.9012 s

In [141]:
def qsort(a):
    b=sorted(a)
    return b

In [148]:
print(*(qsort(a)[:50]))

0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3


In [149]:
%lprun -f qsort qsort(a)

### Total time: 0.0023229 s

In [136]:
from collections import defaultdict
def sort_by_count(a):
    d=defaultdict(int)
    for el in a:
        d[el]+=1
    b=[]
    for i in range(400):
        for j in range(d[i]):
            b.append(i)
    return b
            
    

In [151]:
print(*(sort_by_count(a)[:50]))

0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3


In [152]:
%lprun -f sort_by_count sort_by_count(a)

### Total time: 0.0097716 s

In [157]:
%%writefile compare_sort.py
from memory_profiler import profile
from numpy.random import randint
from collections import defaultdict
import copy

a=randint(0,400,5000)

def sort_buble(a):
    b=copy.deepcopy(a)
    for i in range(len(b)-1):
        for j in range(i+1,len(b)):
            if b[i]>b[j]:
                b[i],b[j]=b[j],b[i]
    return b

def qsort(a):
    b=sorted(a)
    return b


def sort_by_count(a):
    d=defaultdict(int)
    for el in a:
        d[el]+=1
    b=[]
    for i in range(400):
        for j in range(d[i]):
            b.append(i)
    return b


@profile
def compare_sort():
    b=sort_buble(a)
    b1=qsort(a)
    b2=sort_by_count(a)
    return 0
    

if __name__=='__main__':
    compare_sort()

Overwriting compare_sort.py


In [158]:
!python compare_sort.py

Filename: compare_sort.py

Line #    Mem usage    Increment   Line Contents
    32     51.6 MiB     51.6 MiB   @profile
    33                             def compare_sort():
    34     51.7 MiB      0.0 MiB       b=sort_buble(a)
    35     51.8 MiB      0.1 MiB       b1=qsort(a)
    36     51.9 MiB      0.1 MiB       b2=sort_by_count(a)
    37     51.9 MiB      0.0 MiB       return 0




### по быстродействию лучше использовать встроенную сортировку, хотя при больших размерах списка сортировка подсчетом будет быстрее - О(n) против O(n*log(n))  . про память по отчету ничего не понятно, но увеличивать размер списка означает квадратичное увеличение времени работы пузырькового метода сортировки. в принципе и так все понятно.

## 4. Распаковать вложенный список.

In [170]:
x,y,z=3,4,5

In [171]:
a=[[[el for el in range(x)] for i in range(y)] for i in range(z)]

In [172]:
a

[[[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
 [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
 [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
 [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
 [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]]

In [173]:
def unpack(a):
    b=[]
    for i in a:
        for j in i:
            for k in j:
                b.append(k)
    return b

In [175]:
print(*unpack(a))

0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2


In [176]:
import numpy

In [189]:
def myflatten(a):
    return list(numpy.array(a).flatten())

print(*myflatten(a))

0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2


In [183]:
from itertools import chain

In [190]:
def mychain(a):
    return list(chain.from_iterable(chain.from_iterable(a)))
print(*mychain(a))

0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2


In [196]:
x,y,z=300,400,50
a=[[[el for el in range(x)] for i in range(y)] for i in range(z)]

In [197]:
%lprun -f unpack unpack(a)

### Total time: 3.74585 s

In [198]:
%lprun -f myflatten myflatten(a)

### Total time: 0.523447 s

In [199]:
%lprun -f mychain mychain(a)

### Total time: 0.217366 s

In [206]:
%%writefile compare_unpack.py
import numpy
from itertools import chain
from memory_profiler import profile

x,y,z=300,400,50
a=[[[el for el in range(x)] for i in range(y)] for i in range(z)]

def unpack(a):
    b=[]
    for i in a:
        for j in i:
            for k in j:
                b.append(k)
    return b

def myflatten(a):
    return list(numpy.array(a).flatten())

def mychain(a):
    return list(chain.from_iterable(chain.from_iterable(a)))


@profile
def compare_unpack():
    b=unpack(a)
    b1=myflatten(a)
    b2=mychain(a)
    return 0
    

if __name__=='__main__':
    compare_unpack()

Overwriting compare_unpack.py


In [207]:
!python compare_unpack.py

Filename: compare_unpack.py

Line #    Mem usage    Increment   Line Contents
    23    128.9 MiB    128.9 MiB   @profile
    24                             def compare_unpack():
    25    177.2 MiB     48.3 MiB       b=unpack(a)
    26    409.0 MiB    231.8 MiB       b1=myflatten(a)
    27    455.0 MiB     46.0 MiB       b2=mychain(a)
    28    455.0 MiB      0.0 MiB       return 0




### itertools впереди планеты всей. 